In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter
import cv2
from hsi_dataset_api import HsiDataset
from makitorch.dataloaders.HsiDataloader import HsiDataloader

import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json

In [ ]:
PREFIX_INFO_PATH = '/home/rustam/hyperspecter_segmentation/danil_cave/kfolds_data/kfold0'
PATH_DATA = '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data'


pca_explained_variance = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaExplainedVariance_.npy')
pca_mean = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaMean.npy')
pca_components = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaComponents.npy')


test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')
train_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_train.npy')


In [ ]:
def test_augmentation(image, mask):
    image = TF.to_tensor(image)
    #image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    
    mask = torch.squeeze(mask, 0)
    return image, mask


def preprocessing(imgs, masks):
    _images = [np.transpose(image, (1, 2, 0)) for image in imgs]
    _masks = [
        np.expand_dims(
            cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.uint8)
            ,0
        ).astype(np.int64)
        for mask in masks
    ]
    return _images, _masks

In [ ]:
dataset_test = HsiDataloader(
    PATH_DATA, preprocessing=preprocessing, 
    augmentation=test_augmentation, indices=test_indices
)

dataset_train = HsiDataloader(
    PATH_DATA, preprocessing=preprocessing,
    augmentation=test_augmentation, indices=train_indices
)

In [ ]:
val_loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=1)

val_loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=1)

In [ ]:
from tqdm import tqdm

In [ ]:
val_loader = val_loader_train

specter_list = []
target_list = []

for img_s, mask_s in tqdm(val_loader):
    specter_list.append(img_s[0].numpy())
    target_list.append(mask_s[0].numpy())
len(specter_list)

In [ ]:
X_ans = np.transpose(specter_list[0:1].copy(), (0, 2, 3, 1)).reshape(-1, 237)
Ytest = np.asarray(target_list[0:1]).copy().reshape(-1)

X_ans.shape

In [ ]:
np.unique(Ytest)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pca = PCA(2)
reduced = pca.fit_transform(X_ans)
plt.scatter(reduced[:,0],reduced[:,1] ,s=20,c=Ytest[:X_ans.shape[0]],alpha=0.5)
plt.savefig('after_train1.jpg')

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
Z = tsne.fit_transform(X_ans)
plt.scatter(Z[:,0],Z[:,1] ,s=100,c=Ytest[:X_ans.shape[0]],alpha=0.5)
plt.savefig('sas2.jpg')

In [ ]:
plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
import umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(X_ans)
embedding.shape

In [ ]:
plt.scatter(embedding[:,0],embedding[:,1] ,s=100,c=Ytest[:X_ans.shape[0]],alpha=0.5)